In [1]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score,cross_validate,StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import mlflow
import optuna
import sklearn
# Create a new experiment
mlflow.set_experiment("Hyperparameter Tunning Experiment")

#Load the dataframe
data = pd.read_csv('data.csv')
data.head()

#Check the shape of data
data.shape

#check the null values
data.isnull().sum()

#Percentage of null values
(data.isnull().sum()/ len(data)) * 100

### Insight:
### From the above we can infer that null/missing values less than 1% apart from date column.
### we can remove them or replace with mean or median.
### Date column alone we will handle, as it is slightly greater than 5%

# Handle the null values
## We can drop the null values
data.dropna(subset=['Reviewer Name','Review Title','Place of Review','Up Votes','Down Votes','Review text'],
            inplace=True)
# replace the null values of month with ffill / bfill
data['Month'] = data['Month'].fillna(data['Month'].ffill())

# Check the data again
data.head()

data.isnull().sum()

import nltk
nltk.download('stopwords')

from nltk.stem.snowball import stopwords
import re
# Data Preprocessing
stop_words = set(stopwords.words('english'))
def clean_text(text):
  #change all the character to lower case
  text = str(text).lower()
  #Remove the special characters or punctuations
  text = re.sub(r'[^\w\d\s]', ' ',text)
  text = re.sub(r',', ' ',text)
  text = re.sub(r'\s+', ' ',text)
  text = re.sub(r'^\s+|\s+?$', '',text)
  #remove stop words
  text = ''.join(term for term in text.split() if term not in stop_words)
  return text

text_cols = ['Review text']
for col in text_cols:
  data[col]= data[col].apply(clean_text)

nltk.download('wordnet')

## Lemmetization

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
  text = ' '.join(lemmatizer.lemmatize(term,pos='v') for term in str(text).split())
  return text

for col in text_cols:
  data[col] = data[col].apply(lemmatize_text)

# Divide the data into features and target
X = data['Review text']
data['Sentiment'] = data['Ratings'].apply(lambda x:'positive' if x>=4 else 'Negative')
y = data.Sentiment

# Divide the data into train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## TF-IDF
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer(max_features=5000,ngram_range=(1, 2))
# X_train_tfidf = tfidf.fit_transform(X_train)
# X_test_tfidf = tfidf.transform(X_test)

from sklearn.pipeline import Pipeline
pipeline = Pipeline([('tfidf',TfidfVectorizer()),
 ('clf',LogisticRegression())])

#pipeline = Pipeline(
#     [
#         ('Scaler', StandardScaler()),
#         ('Model', KNeighborsClassifier())
#     ]
# )

def objective_rf(trial):

    # Random Forest hyperparameters
    pipeline.set_params(
    tfidf=TfidfVectorizer(max_features=4800, ngram_range=(1,2), min_df=5, stop_words='english'),
    clf=RandomForestClassifier(
        n_estimators=trial.suggest_int('n_estimators', 100, 500, step=50),
        # ... other suggested params ...
        random_state=42,
        n_jobs=-1
    ))

    skf = StratifiedKFold(n_splits=5, shuffle=True)
    score = cross_val_score(
        pipeline,
        X_train,
        y_train,
        scoring='accuracy',
        cv=skf
    ).mean()

    return score

def objective_lr(trial):

    # Random Forest hyperparameters
    pipeline.set_params(
    tfidf=TfidfVectorizer(max_features=4800, ngram_range=(1,2), min_df=5, stop_words='english'),
    clf=LogisticRegression(
          # Regularization strength: lower values mean stronger regularization
          C=trial.suggest_float('C', 1e-5, 100, log=True),
          # Solver choice: 'liblinear' and 'lbfgs' are standard for text
          solver=trial.suggest_categorical('solver', ['liblinear', 'lbfgs']),
          max_iter=1000,
          class_weight='balanced',
          random_state=42
        ))

    skf = StratifiedKFold(n_splits=5, shuffle=True)
    score = cross_val_score(
        pipeline,
        X_train,
        y_train,
        scoring='accuracy',
        cv=skf,
        n_jobs = -1
    ).mean()

    return score

def objective_multinb(trial):

    # Random Forest hyperparameters
    pipeline.set_params(
    tfidf=TfidfVectorizer(max_features=4800, ngram_range=(1,2), min_df=5, stop_words='english'),
    clf=MultinomialNB(
            # Alpha is the smoothing parameter (0.0 means no smoothing)
            alpha=trial.suggest_float('alpha', 1e-3, 10.0, log=True),
            # fit_prior determines whether to learn class prior probabilities
            fit_prior=trial.suggest_categorical('fit_prior', [True, False])
        ))

    skf = StratifiedKFold(n_splits=5, shuffle=True)
    score = cross_val_score(
        pipeline,
        X_train,
        y_train,
        scoring='accuracy',
        cv=skf,
        n_jobs = -1
    ).mean()
    return score

def objective_svc(trial):

    # Random Forest hyperparameters
    pipeline.set_params(
    tfidf=TfidfVectorizer(max_features=4800, ngram_range=(1,2), min_df=5, stop_words='english'),
    clf=SVC(
            # Regularization parameter
            C=trial.suggest_float('C', 0.1, 10.0, log=True),
            # Kernel type
            kernel=trial.suggest_categorical('kernel', ['linear', 'rbf']),
            # Kernel coefficient (only for 'rbf')
            gamma=trial.suggest_categorical('gamma', ['scale', 'auto']),
            # Useful for imbalanced text classes
            class_weight='balanced',
            random_state=42,
            max_iter=2000 # Prevents infinite loops on non-converging trials
        )
    )

    skf = StratifiedKFold(n_splits=5, shuffle=True)
    score = cross_val_score(
        pipeline,
        X_train,
        y_train,
        scoring='accuracy',
        cv=skf,
        n_jobs = -1
    ).mean()
    return score

with mlflow.start_run(run_name="study") as run:
    # Log the experiment settings
    n_trials = 30
    mlflow.log_param("n_trials", n_trials)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective_svc, n_trials=n_trials)

    # Log the best trial and its run ID
    mlflow.log_params(study.best_trial.params)
    mlflow.log_metrics({"best_error": study.best_value})
    if best_run_id := study.best_trial.user_attrs.get("run_id"):
        mlflow.log_param("best_child_run_id", best_run_id)

#!pip install optuna-integration[mlflow]

from optuna.integration.mlflow import MLflowCallback
import time
import joblib
import os

# Model Building

results = {}
model_dict = {}

objectives = {
    'logistic regression':objective_lr,
    'svc':objective_svc,
    'RandomForest':objective_rf,
    'multinb':objective_multinb,
}
for model_name,obj_fn in objectives.items():
    print(f"\n--- Optimizing {model_name} ---")

    # # Disable autologging once
    # mlflow.sklearn.autolog(disable = True)

    mlflow_cb = MLflowCallback(
        tracking_uri=None,              # Auto-detect (str/None)
        metric_name="cv_accuracy",      # Primary metric (str)
        mlflow_kwargs={                 # **MLflow start_run() kwargs**
            "nested": True              # Child runs under parent
        }
    )

    # Create Optuna study
    study = optuna.create_study(direction="maximize")

    # Train the final model
    start_fit = time.time()
    study.optimize(obj_fn, n_trials=20, callbacks=[mlflow_cb])
    fit_time = time.time() - start_fit

    print(f"Best CV accuracy for {model_name}: {study.best_value:.4f}")
    best_params = study.best_params
    results[model_name] = {"best_params": best_params, "best_cv_accuracy": study.best_value}

    # Fit the pipeline with the best parameters
    if model_name == "logistic regression":
        # 1. Update TF-IDF (The Transformer)
        pipeline.set_params(
            tfidf=TfidfVectorizer(max_features=4800, ngram_range=(1,2), stop_words='english'),

            # 2. Update Logistic Regression (The Classifier)
            # Use 'clf__' to match your pipeline's second step
            clf__C=best_params["C"],
            clf__solver=best_params.get("solver", "liblinear"), # Default to liblinear for text
            clf__max_iter=1000,
            clf__class_weight='balanced',
            clf__random_state=42
        )
    elif model_name == "RandomForest":
        # 1. Update TF-IDF (The Transformer)
        # We use best_params if you tuned TF-IDF, otherwise use your fixed settings
        pipeline.set_params(
            tfidf=TfidfVectorizer(max_features=4800, ngram_range=(1,2), stop_words='english'),

            # 2. Update Random Forest (The Classifier)
            # Ensure 'clf__' matches the name of your second step in the Pipeline
            clf__n_estimators=best_params["n_estimators"],
            clf__n_jobs=-1,
            clf__random_state=42
        )
    elif model_name == "svc":
        # 1. Update TF-IDF (The Transformer)
        pipeline.set_params(
            tfidf=TfidfVectorizer(max_features=4800, ngram_range=(1,2), stop_words='english')
        )

        # 2. Update SVC (The Classifier)
        # We use .get() for gamma to avoid errors if it wasn't suggested in a linear trial
        pipeline.set_params(
            clf__C=best_params["C"],
            clf__kernel=best_params["kernel"],
            clf__gamma=best_params["gamma"],
            clf__class_weight='balanced',
            clf__random_state=42,
            clf__max_iter=2000
        )
    elif model_name == "multinb":
        # 1. Update TF-IDF
        pipeline.set_params(
            tfidf=TfidfVectorizer(max_features=4800, ngram_range=(1,2), stop_words='english')
        )
        # 2. Update MultinomialNB (The Classifier)
        # alpha is the primary hyperparameter for Naive Bayes
        pipeline.set_params(
            clf__alpha=best_params["alpha"],
            clf__fit_prior=best_params.get("fit_prior", True)
        )



    # # Enable autologging once
    # mlflow.sklearn.autolog()

    # Train the final model
    pipeline.fit(X_train, y_train)

    # Evaluate on test data
    start_test = time.time()
    y_pred = pipeline.predict(X_test)
    test_time = time.time() - start_test

    train_acc = pipeline.score(X_train, y_train)
    test_acc = accuracy_score(y_test, y_pred)

    print(f"{model_name} Training Accuracy: {train_acc:.4f}, Testing Accuracy: {test_acc:.4f}")
    print(f"{model_name} Fit Time: {fit_time:.2f}s, Test Time: {test_time:.2f}s")

    # Save model manually to track model size
    model_path = f"{model_name}_final_model.pkl"
    joblib.dump(pipeline, model_path)
    model_size = os.path.getsize(model_path)
    for i, model in enumerate(objectives.keys()):
        model_dict[model] = i

    # for i, scaler_type in enumerate(['standard', 'minmax']):
    #     scaler_dict[scaler_type] = i

    mlflow.log_metric(f"model_id", model_dict[model_name])
   # mlflow.log_metric(f"Scalar_id", scaler_dict[scaler_type])
    mlflow.log_metric(f"train_accuracy", train_acc)
    mlflow.log_metric(f"test_accuracy", test_acc)
    mlflow.log_metric(f"train_time", fit_time)
    mlflow.log_metric(f"test_time", test_time)
    mlflow.log_metric(f"model_size", model_size)
    mlflow.sklearn.log_model(pipeline, name=f"{model_name}Sentimental_Analysis") #, registered_model_name = f"Iris_{model_name}_Best")
    os.remove(model_path)

    results[model_name].update({
        "train_accuracy": train_acc,
        "test_accuracy": test_acc,
        "fit_time": fit_time,
        "test_time": test_time,
        "model_size_bytes": model_size
    })
    mlflow.end_run()

# Summary
print("\n--- Summary ---")
for model_name, res in results.items():
    print(f"{model_name}: CV Acc={res['best_cv_accuracy']:.4f}, Train Acc={res['train_accuracy']:.4f}, "
          f"Test Acc={res['test_accuracy']:.4f}, Fit Time={res['fit_time']:.2f}s, "
          f"Model Size={res['model_size_bytes']} bytes")


2026/02/04 19:48:55 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/04 19:48:55 INFO mlflow.store.db.utils: Updating database tables
2026/02/04 19:48:55 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/04 19:48:55 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/04 19:48:55 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/04 19:48:55 INFO alembic.runtime.migration: Will assume non-transactional DDL.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dathu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dathu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2026/02/04 19:49:00 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git ex

(6774,)
(1694,)
(6774,)
(1694,)


[I 2026-02-04 19:49:04,558] Trial 0 finished with value: 0.6838090773817636 and parameters: {'C': 8.471575215708652, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.6838090773817636.
[I 2026-02-04 19:49:06,941] Trial 1 finished with value: 0.8191614840816059 and parameters: {'C': 2.2429853397714545, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 0 with value: 0.6838090773817636.
[I 2026-02-04 19:49:07,384] Trial 2 finished with value: 0.807646933781007 and parameters: {'C': 0.8040784128361741, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.6838090773817636.
[I 2026-02-04 19:49:07,838] Trial 3 finished with value: 0.6838090773817636 and parameters: {'C': 0.12476333851668513, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.6838090773817636.
[I 2026-02-04 19:49:08,325] Trial 4 finished with value: 0.4190124654570032 and parameters: {'C': 6.020339993669527, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 4 with value: 0.41901246545


--- Optimizing logistic regression ---


[I 2026-02-04 19:49:17,839] Trial 1 finished with value: 0.5662833098050332 and parameters: {'C': 4.55191423135366, 'solver': 'lbfgs'}. Best is trial 1 with value: 0.5662833098050332.
[I 2026-02-04 19:49:18,087] Trial 2 finished with value: 0.4918776673734241 and parameters: {'C': 0.0014334013584948982, 'solver': 'liblinear'}. Best is trial 1 with value: 0.5662833098050332.
[I 2026-02-04 19:49:18,316] Trial 3 finished with value: 0.5138737756653785 and parameters: {'C': 0.021782477223648237, 'solver': 'liblinear'}. Best is trial 1 with value: 0.5662833098050332.
[I 2026-02-04 19:49:18,520] Trial 4 finished with value: 0.4938004109730906 and parameters: {'C': 0.0008139681097236804, 'solver': 'liblinear'}. Best is trial 1 with value: 0.5662833098050332.
[I 2026-02-04 19:49:18,756] Trial 5 finished with value: 0.5618548294788709 and parameters: {'C': 0.20216745804971892, 'solver': 'lbfgs'}. Best is trial 1 with value: 0.5662833098050332.
[I 2026-02-04 19:49:18,917] Trial 6 finished with v

...

Best CV accuracy for RandomForest: 0.8246
RandomForest Training Accuracy: 0.9553, Testing Accuracy: 0.8200
RandomForest Fit Time: 105.29s, Test Time: 0.18s


C:\Users\dathu\AppData\Local\Temp\ipykernel_36948\1796424395.py:262: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflow_cb = MLflowCallback(
[I 2026-02-04 19:51:41,001] A new study created in memory with name: no-name-c7f5c610-56f9-43ae-9ad7-e04c06d0c0ea
[I 2026-02-04 19:51:41,137] Trial 0 finished with value: 0.5608199839753197 and parameters: {'alpha': 0.2436227563371249, 'fit_prior': False}. Best is trial 0 with value: 0.5608199839753197.
2026/02/04 19:51:41 INFO mlflow.tracking.fluent: Experiment with name 'no-name-c7f5c610-56f9-43ae-9ad7-e04c06d0c0ea' does not exist. Creating a new experiment.



--- Optimizing multinb ---


[I 2026-02-04 19:51:41,392] Trial 1 finished with value: 0.8228516299934048 and parameters: {'alpha': 0.16116531280213256, 'fit_prior': True}. Best is trial 1 with value: 0.8228516299934048.
[I 2026-02-04 19:51:41,594] Trial 2 finished with value: 0.5599382995307057 and parameters: {'alpha': 0.002984258449938245, 'fit_prior': False}. Best is trial 1 with value: 0.8228516299934048.
[I 2026-02-04 19:51:41,812] Trial 3 finished with value: 0.8218182016384418 and parameters: {'alpha': 0.018771461990897496, 'fit_prior': True}. Best is trial 1 with value: 0.8228516299934048.
[I 2026-02-04 19:51:42,021] Trial 4 finished with value: 0.5612610442204865 and parameters: {'alpha': 0.05042918892837878, 'fit_prior': False}. Best is trial 1 with value: 0.8228516299934048.
[I 2026-02-04 19:51:42,215] Trial 5 finished with value: 0.8235898553963382 and parameters: {'alpha': 0.14343384219062166, 'fit_prior': True}. Best is trial 5 with value: 0.8235898553963382.
[I 2026-02-04 19:51:42,460] Trial 6 finis

Best CV accuracy for multinb: 0.8242
multinb Training Accuracy: 0.8062, Testing Accuracy: 0.7898
multinb Fit Time: 4.28s, Test Time: 0.01s

--- Summary ---
logistic regression: CV Acc=0.8192, Train Acc=0.9504, Test Acc=0.8093, Fit Time=4.59s, Model Size=243572 bytes
svc: CV Acc=0.8172, Train Acc=0.1938, Test Acc=0.2102, Fit Time=11.44s, Model Size=356752 bytes
RandomForest: CV Acc=0.8246, Train Acc=0.9553, Test Acc=0.8200, Fit Time=105.29s, Model Size=41942981 bytes
multinb: CV Acc=0.8242, Train Acc=0.8062, Test Acc=0.7898, Fit Time=4.28s, Model Size=322193 bytes


In [2]:
import joblib
joblib.dump(pipeline, "multinb_sentiment_model.pkl")

['multinb_sentiment_model.pkl']